# Notebook Usage

<ul>
<li>Create an intact weekly level timeseries of the prices of products in /data exels
<li>This data will be visualized in the frontend react app
</ul>

#### Processing logic

1. Load all excels into a single dataframe
2. Create time periods (weeks, months) from start of data to end of data
3. For each product, (EAN-koodi) generate the time series
* Max price during time period
4. Make test visualization before frontend functionality

* Consider product grouping, could be useful e.g. if product has only slightly changed


In [1]:
import os
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *


os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"


In [2]:
# Load in the excels using pandas

file_location = "..\data"

pdf_raw = pd.DataFrame()

for file_name in os.listdir(file_location):
    if file_name[-4:] == 'xlsx':
        print(file_name)
        df_one_file = pd.read_excel(f"{file_location}\{file_name}", skiprows=8)
        df_one_file["source"] = file_name
        pdf_raw = pd.concat([pdf_raw, df_one_file])
    

ostotietoraportti_aluoto.xlsx
ostotietoraportti_jluoto.xlsx


In [3]:
# Create spark session
# spark = (SparkSession.builder
#          .appName("price_analysis")
#          .config("spark.sql.shuffle.partitions", "1")  # Reduce shuffle partitions for operations like join, groupBy
#          .config("spark.default.parallelism", "4")  # Adjust parallelism; set to a reasonable number for your machine
#          .getOrCreate())

from delta import *
import pyspark

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.hadoop.hadoop.home.dir", "C:/hadoop") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")# \
    # .config("spark.default.parallelism", "4") \
    # .config("spark.sql.shuffle.partitions", "1")


spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [4]:
# Create spark dataframe
raw_data = spark.createDataFrame(pdf_raw)
raw_data.show(5)

+-------------------+----------+---------------+--------------------+--------------------+-------------+-------------------------+----------------------------+-------------+------------+-------+--------------------+
|         PÄIVÄMÄÄRÄ|KELLONAIKA|    TOIMIPAIKKA|          TUOTERYHMÄ|           TUOTENIMI|    EAN-KOODI|Yksikköhinta € verollinen|Ostetut tuotteet (paino/kpl)|Myyntihinta €|Alennukset €|OSTOT €|              source|
+-------------------+----------+---------------+--------------------+--------------------+-------------+-------------------------+----------------------------+-------------+------------+-------+--------------------+
|2021-06-30 00:00:00|     15:08|PRISMA HYVINKÄÄ|Mehu- ja juoma-an...|10 x Rainbow päär...|6415712500894|                     2.15|                         1.0|         2.15|         0.0|   2.15|ostotietoraportti...|
|2021-06-30 00:00:00|     15:08|PRISMA HYVINKÄÄ|Mehu- ja juoma-an...|10 x Rainbow vade...|6415712500887|                     2.15|      

In [5]:
# Create the timestamps as the baseline for price time series

mindate = raw_data.agg(min('PÄIVÄMÄÄRÄ')).first()[0]
maxdate = raw_data.agg(max('PÄIVÄMÄÄRÄ')).first()[0]

date_range = pd.date_range(start=mindate, end=maxdate)

# Spark expect a list of iterables = list of lists or list of tuples
date_list = [(date.to_pydatetime(),) for date in date_range]

# Convert pandas date range to Spark DataFrame
schema = StructType([
    StructField("Date", TimestampType(), False)
     ])

dates_df = spark.createDataFrame(date_list, schema=schema)

dates_df.show(3)

+-------------------+
|               Date|
+-------------------+
|2019-01-01 00:00:00|
|2019-01-02 00:00:00|
|2019-01-03 00:00:00|
+-------------------+


In [6]:
def price_history_for_product(product_transactions:DataFrame, dates_df:DataFrame) -> DataFrame:
    """
    Input: Spark dataframe
    Output: Spark dataframe

    Fills the gaps in the transactions dataframe
    """
    # Get max price for each day (if multiple rows for one day)
    product_daily_prices = product_transactions.groupby('PÄIVÄMÄÄRÄ').agg(max('Yksikköhinta € verollinen').alias('Yksikköhinta € verollinen'))
    product_daily_prices = product_daily_prices.withColumn("EAN-KOODI", lit(ean_code))
    # product_daily_prices.show(3)

    # left join product daily price with the date range
    product_daily_prices_full = dates_df.join(product_daily_prices, product_daily_prices.PÄIVÄMÄÄRÄ==dates_df.Date, how='left')
    # product_daily_prices_full.show(10)

    # Transform spark dataframe to pandas-on-spark -dataframe to allow ffill -logic
    ps_product_daily_prices_full = product_daily_prices_full.pandas_api()
    # display(ps_product_daily_prices_full.head(10))
    ps_product_daily_prices_full = ps_product_daily_prices_full.sort_values(by='Date')
    ps_product_daily_prices_full = ps_product_daily_prices_full.ffill()
    # display(ps_product_daily_prices_full.tail(5))

    # Transform pandas-on-spark -dataframe back to spark dataframe
    product_daily_prices_full_final = ps_product_daily_prices_full.to_spark()

    return product_daily_prices_full_final


In [7]:
# Get all products

all_products = raw_data.select('EAN-KOODI').distinct()
print(f"Product count: {all_products.count()}")


Product count: 2962


In [8]:
all_ean_codes = [row['EAN-KOODI'] for row in all_products.collect()]

In [9]:
# Get max price for each day for a product

price_history_all_products = None
ean_codes = all_ean_codes[:2]

for ean_code in ean_codes:
    # Filter with current product
    product_transactions = raw_data.filter(col('EAN-KOODI') == ean_code)
    # Get price history for current product
    one_product_history = price_history_for_product(product_transactions, dates_df)

    # Append data into the price_history_all_products Spark DataFrame
    if price_history_all_products is None:
        price_history_all_products = one_product_history
    else:
        price_history_all_products = price_history_all_products.union(one_product_history)



C:\Users\Antti Luoto\IdeaProjects\data-visualization\venv-data-visualization\Lib\site-packages\pyspark\pandas\utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [10]:

def format_column_name(col_name):
    # Split the column name by spaces
    parts = col_name.split()
    # Capitalize the first letter of each word except the first one
    new_parts = [parts[0].lower()] + [word.capitalize() for word in parts[1:]]
    # Join parts without spaces
    new_col_name = ''.join(new_parts)
    return new_col_name

# Apply the function to each column name and rename the columns
for col_name in price_history_all_products.columns:
    new_col_name = format_column_name(col_name)
    price_history_all_products = price_history_all_products.withColumnRenamed(col_name, new_col_name)


In [11]:
# price_history_all_products.write.format("delta").mode("overwrite").save("..\data\price_history_all_products_delta")

# Coalesce the DataFrame to ensure only one part-file is created
price_history_all_products.coalesce(1) \
    .write.format('csv') \
    .mode('overwrite') \
    .option('header', 'true') \
    .save('test.csv')

Py4JJavaError: An error occurred while calling o409.save.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.getAllCommittedTaskPaths(FileO

In [ ]:
# Coalesce the DataFrame to ensure only one part-file is created
# price_history_all_products.coalesce(1).write.format('csv').option('header', 'true').save('test.csv')


import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.plot(ps_product_daily_prices_full['Date'], ps_product_daily_prices_full['Yksikköhinta € verollinen'])
    